In [1]:
import pandas as pd
import Levenshtein

In [2]:
words = pd.read_csv('words.csv')

In [3]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教師
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [4]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [5]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [6]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.7
    if pd.isna(correct_answer) or user_answer == '':
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [7]:
shuffled_sample = shuffled_words.sample(n=40)
shuffled_sample

,Français,Hiragana,Kanji
96,mâcher / mordre,かみます,NaN
255,panne,こしょう,故障
315,agenda,てちょう,手帳
6,accueil,うけつけ,受付
128,laisser tel quel / comme ça,そのままにします,NaN
239,réparer,しゅうりします,修理します
178,brûler,もえます,燃えます
11,chef de département,ぶちょう,部長
31,[arbre] (se) casser,おれます,折れます
100,à peu près,だいたい,NaN


In [8]:
seen_fr = []
correct_count = 0
score_fr = 0
shuffled_sample_jp = shuffled_sample.sample(frac=1).reset_index(drop=True)

for i in range(len(shuffled_sample_jp)):
    if shuffled_sample_jp['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"Comment dire {shuffled_sample_jp['Français'].iloc[i]} en hiragana ?")
        seen_fr.append(shuffled_sample_jp['Français'].iloc[i])

        if is_similar_japanese_answer(ask_h, shuffled_sample_jp['Hiragana'].iloc[i]):
            kanji_value = shuffled_sample_jp['Kanji'].iloc[i]
            if pd.notna(kanji_value):
                ask_k = input("Et en kanji ?")
                if is_similar_japanese_answer(ask_k, kanji_value):
                    correct_count += 1
                    score_fr += 1
                    print('Correct !')
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
                else:
                    print(f"Incorrect ! La bonne réponse est : {kanji_value}")
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
            else:
                correct_count += 1
                score_fr += 1
                print('Correct !')
                print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
        else:
            print(f"Incorrect ! La bonne réponse est : {shuffled_sample_jp['Hiragana'].iloc[i]}")
            print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')

if correct_count == len(shuffled_sample_jp):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")
else:
    print(f"Vous avez répondu correctement à {score_fr} des {len(shuffled_sample_jp)} questions.")

Comment dire vivre en hiragana ?すみます
Incorrect ! La bonne réponse est : せいかつします
Bonnes réponses : 0/40
Comment dire mâcher / mordre en hiragana ?かみます
Correct !
Bonnes réponses : 1/40
Comment dire boîte en métal / canette en hiragana ?
Incorrect ! La bonne réponse est : かん
Bonnes réponses : 1/40
Comment dire parking en hiragana ?ちゅうしゃじょう
Et en kanji ?駐車場
Correct !
Bonnes réponses : 2/40
Comment dire apprendre (par quelqu'un) en hiragana ?ならいます
Et en kanji ?習います
Correct !
Bonnes réponses : 3/40
Comment dire programme / planning en hiragana ?よていひょう
Et en kanji ?予定表
Correct !
Bonnes réponses : 4/40
Comment dire à peu près en hiragana ?ほとんど
Incorrect ! La bonne réponse est : だいたい
Bonnes réponses : 4/40
Comment dire programme / émission en hiragana ?ばんぐみ
Et en kanji ?番組
Correct !
Bonnes réponses : 5/40
Comment dire [arbre] (se) casser en hiragana ?えだ
Incorrect ! La bonne réponse est : おれます
Bonnes réponses : 5/40
Comment dire milieu / centre en hiragana ?ちゅうおう
Incorrect ! La bonne réponse est

In [9]:
seen_k=[]
correct_count = 0
score_jp = 0
shuffled_sample_fr = shuffled_sample.sample(frac=1).reset_index(drop=True)
for i in range(len(shuffled_sample_fr)):
    kanji = shuffled_sample_fr['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Que veut dire ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample_fr['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            score_jp += 1
            print(f"Correct ! C'était bien {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr["Kanji"].dropna())}')
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr["Kanji"].dropna())}')

if correct_count == len(shuffled_sample_fr):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")
else:
    print(f"Vous avez répondu correctement à {score_jp} des {len(shuffled_sample_fr['Kanji'].dropna())} questions.")

Que veut dire ce kanji : 手帳?
Incorrecte ! La bonne réponse était : agenda
Bonnes réponses : 0/34
Que veut dire ce kanji : 本棚?étagère
Correct ! C'était bien étagère à livre
Bonnes réponses : 1/34
Que veut dire ce kanji : 細かいお金?petit monnaire
Correct ! C'était bien petite monnaie
Bonnes réponses : 2/34
Que veut dire ce kanji : 汚れます?se salir
Correct ! C'était bien [vêtement] se salir
Bonnes réponses : 3/34
Que veut dire ce kanji : 予定表?
Incorrecte ! La bonne réponse était : programme / planning
Bonnes réponses : 3/34
Que veut dire ce kanji : 燃えます?se bûler
Incorrecte ! La bonne réponse était : brûler
Bonnes réponses : 3/34
Que veut dire ce kanji : 開きます?s'ouvrir
Correct ! C'était bien [porte] s'ouvrir
Bonnes réponses : 4/34
Que veut dire ce kanji : 熱?fievre
Correct ! C'était bien fièvre
Bonnes réponses : 5/34
Que veut dire ce kanji : 番組?emission
Incorrecte ! La bonne réponse était : programme / émission
Bonnes réponses : 5/34
Que veut dire ce kanji : 部長?chef de département
Correct ! C'était 